In [14]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Unigrams to Generate a Vocabulary File**

## **Goal**: Make a list of the _n_ most common unigrams for later use filtering multigrams

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

### Download unigrams
Here, I'm using the `download_ngrams` module to fetch unigrams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). Note the `repo_release_id` and `repo_corpus_id` parameters; these tell the module which ngram corpus and release to download.

In [18]:
download_ngram_files(
    ngram_size=1,
    ngram_type='tagged',
    repo_release_id='20200217',
    repo_corpus_id='eng-fiction',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True
)

Start Time:                2025-04-07 16:13:17.441931

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/eng-fiction-1-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download
File index range:          0 to 0
File URLs available:       1
File URLs to use:          1
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/1-00000-of-00001.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/1-00000-of-00001.gz
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  True



Downloading:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-04-07 16:13:50.611621
Total runtime:             0:00:33.169690



### Convert files from TXT to JSONL
This module converts the original unigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

In [19]:
convert_to_jsonl_files(
    ngram_size=1,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-04-07 16:13:54.730968

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert
File index range:          0 to 0
Files available:           1
Files to use:              1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download/1-00000-of-00001.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download/1-00000-of-00001.txt.lz4
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Converting:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-04-07 16:15:48.691505
Total runtime:             0:01:53.960537



### Make unigrams all lowercase
This module lowercases all characters in the unigrams. Most use cases benefit from this.

In [21]:
lowercase_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-04-07 16:17:03.758827

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase
File index range:          0 to 0
Files available:           1
Files to use:              1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert/1-00000-of-00001.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert/1-00000-of-00001.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lowercasing:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-04-07 16:17:55.339178
Total runtime:             0:00:51.580351



### Lemmatize the unigrams
This module lemmatizes the unigrams—that is, reduces them to their base forms. This is desirable for most use cases. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. My code uses the [NLTK Lemmatizer](https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet), which requires requires POS-tagged unigrams. The tags are discarded after lemmatization as they're no longer useful, saving space.

In [22]:
lemmatize_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-04-07 16:18:05.282307

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize
File index range:          0 to 0
Files available:           1
Files to use:              1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase/1-00000-of-00001.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase/1-00000-of-00001.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lemmatizing:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-04-07 16:19:10.595392
Total runtime:             0:01:05.313085



### Filter the unigrams
This module removes tokens that provide little information about words' semantic context: those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`).

You can also specify `min_tokens`—the minumum length of a retained ngram after filtering its tokens. This is mainly intended for use when processing multigrams. However, it's still good to specify `min_tokens=1` for unigrams, as it completely discards the data for any unigram violating our criteria. Empty output files will be deleted.

In [24]:
filter_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=1,
    compress=True,
    overwrite=True,
    replace_unk=True,
    delete_input=True
)

Start Time:                   2025-04-07 16:19:45.546036

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter
File index range:             0 to 0
Files available:              1
Files to use:                 1
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize/1-00000-of-00001.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize/1-00000-of-00001.jsonl.lz4
Ngram size:                   1
Number of workers:            48
Compress output files:        True
Overwrite existing files:     True
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:          

Filtering:   0%|          | 0/1 [00:00<?, ?files/s]


Filtering Results (Dropped)
Stopword tokens:              2857 
Short-word tokens:            9544 
Tokens with numerals:         239217 
Tokens with non-alpha chars:  162326
Out-of-vocab tokens:          0
Entire ngrams:                413944 

End Time:                  2025-04-07 16:20:41.832845
Total runtime:             0:00:56.286809



### Sort and combine the unigram files
This modules creates a single, fully-sorted unigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

In [26]:
sort_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    workers=12,
    sort_key='ngram',
    compress=True,
    overwrite=True,
    sort_order='ascending',
    delete_input=True
)

Start Time:                2025-04-07 16:21:30.422355

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-merged.jsonl.lz4
Files available:           1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter/1-00000-of-00001.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter/1-00000-of-00001.jsonl.lz4
Files to use:              1
Ngram size:                1
Number of workers:         12
Compress output files:     True
Overwrite existing files:  True
Sort key:                  ngram
Sort order:    

Sorting:   0%|          | 0/1 [00:00<?, ?files/s]

Merging complete. Final file: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-merged.jsonl.lz4

End Time:                  2025-04-07 16:22:34.028715
Total runtime:             0:01:03.606360



### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [30]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/'
        '1gram_files/6corpus/1gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

Lines: 2075511line [00:25, 81017.24line/s]

The file is sorted.


### Consolidate duplicate unigrams
This module consolidates the sorted unigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime with compression:  0:07:33.662163`

In [31]:
consolidate_duplicate_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    lines_per_chunk=500000,
    compress=True,
    overwrite=True
)

Start Time:                2025-04-07 16:26:12.724244

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/temp_chunks
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Created and Sorted: 5 chunks
Merged: 5 chunks


End Time:                  2025-04-07 16:27:05.051701
Total runtime:             0:00:52.327457



### Index unigrams and create vocabulary file
Most use cases will require an indexed list of "valid" (i.e., reasonably common) vocabulary words. This indexing script serves the dual functions of (1) mapping each unigram to an index number (saved in `/6corpus/1gram-corpus-indexed.jsonl`) and (2) culling this file into a vocabulary list consisting of the _n_ most frequent unigrams (saved in `6corpus/1gram-corpus-vocab_list_match.txt`). The vocabulary file provides a critical means of filtering excessively rare words out of the corpus. Unlike files upstream in the workflow, the vocabulary files are not large and don't need to be compressed.

In [32]:
index_and_create_vocab_files(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    vocab_n=80000
)

Start Time:                2025-04-07 16:27:33.930317

Indexing Info
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Indexed file:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-indexed.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Vocabulary Info
Vocab size (top N):        80000
Match File:                /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus-vocab_list_match.txt
Lookup File:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus-vocab_list_lookup.jsonl



Chunking:   0%|          | 0/1125984 [00:00<?, ?lines/s]

Sorting:   0%|          | 0/12 [00:00<?, ?chunks/s]

Merging:   0%|          | 0/1125984 [00:00<?, ?lines/s]

Indexing:   0%|          | 0/1125984 [00:00<?, ?lines/s]


Indexed 1125984 lines.
Final indexed file: 1gram-corpus-indexed.jsonl.lz4
Created vocab_list_match and vocab_list_lookup files for top 80000 ngrams.

End Time:                  2025-04-07 16:28:41.158170
Total runtime:             0:01:07.227853



### Verify indexing [OPTIONAL]
We can verify that the final indexed file looks right.

In [37]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/'
        '1gram_files/6corpus/1gram-corpus-indexed.jsonl.lz4'
    ),
    start_line=40000,
    end_line=40000,
    parse_json=True
)

Line 40000: {'ngram': 'towelling', 'freq_tot': 36694, 'doc_tot': 25173, 'freq': {'1765': 1, '1812': 1, '1819': 1, '1823': 2, '1830': 2, '1833': 1, '1834': 4, '1836': 2, '1837': 4, '1840': 2, '1843': 1, '1846': 1, '1848': 4, '1850': 2, '1851': 3, '1852': 4, '1853': 3, '1854': 2, '1857': 6, '1858': 2, '1859': 8, '1860': 2, '1861': 22, '1862': 8, '1863': 3, '1864': 5, '1865': 33, '1866': 7, '1867': 6, '1868': 7, '1869': 6, '1870': 7, '1871': 12, '1872': 10, '1873': 2, '1874': 15, '1875': 10, '1876': 13, '1877': 7, '1878': 7, '1879': 13, '1880': 14, '1881': 8, '1882': 3, '1883': 20, '1884': 4, '1885': 10, '1886': 12, '1887': 7, '1888': 8, '1889': 7, '1890': 5, '1891': 15, '1892': 9, '1893': 4, '1894': 6, '1895': 6, '1896': 19, '1898': 20, '1899': 44, '1900': 23, '1901': 15, '1902': 19, '1903': 12, '1904': 9, '1905': 29, '1906': 19, '1907': 19, '1908': 24, '1909': 12, '1910': 24, '1911': 18, '1912': 18, '1913': 6, '1914': 14, '1915': 9, '1916': 8, '1917': 22, '1918': 12, '1919': 17, '1920':

## Next Steps
If you've gotten this far, you're ready to start pre-processing multigrams using the `workflow_multigrams.ipynb` notebook!